# MongoDB

---

**Источники:**

[Большой туториал по MongoDB](https://medium.com/@Merrick_krg/%D0%B1%D0%BE%D0%BB%D1%8C%D1%88%D0%BE%D0%B9-%D1%82%D1%83%D1%82%D0%BE%D1%80%D0%B8%D0%B0%D0%BB-%D0%BF%D0%BE-mongodb-7e2f9e17f0c7)

[MongoDB vs MySQL: A Comparative Study on Databases](https://www.simform.com/mongodb-vs-mysql-databases)

[Academy 3T](https://studio3t.com/academy/)

[Robo 3T GUI](https://robomongo.org/download)

---

## Терминология

**MongoDB** — документоориентированная система управления базами данных, не требующая описания схемы таблиц.

Считается одним из классических примеров NoSQL-систем. 

Использует JSON-подобные документы и схему базы данных.

Написана на языке C++. 

<center><img src="images/MongoDB_database_colection.png"/></center>

<center><img src="images/rdbms_vs_mongodb.png"/></center>

<center><img src="images/mongodb_and_rdbms_schema.png"/></center>

**MongoDB подходит для следующих применений**:
- хранение и регистрация событий;
- системы управления документами и контентом;
- электронная коммерция;
- игры;
- данные мониторинга, датчиков;
- мобильные приложения;
- хранилище операционных данных веб-страниц (например, хранение комментариев, рейтингов, профилей пользователей, сеансы пользователей).

---

## Подготовка окружения

### Установка MongoDB

[MongoDB Community Server](https://www.mongodb.com/try/download/community)

[Robo 3T: the hobbyist GUI](https://robomongo.org/download)

In [1]:
!which pip

In [ ]:
!conda install pymongo -y

Solving environment: - 

---

## MongoDB Типы данных

...

---

## MongoDB + Python

In [ ]:
import pymongo
from pymongo import MongoClient

pymongo.__version__

---

### Подключение к базе данных

In [ ]:
# данные для подключения к БД
host='localhost'
port=3306

In [ ]:
client = MongoClient(f'mongodb://{host}:{port}/') # MongoClient(host, port)
client

In [ ]:
# db = client['test-database']
db = client['test-database']
db

In [ ]:
# collection = db['test_collection']
collection = db.test_collection
collection

In [ ]:
import datetime

post = {"author": "Mike",
        "text": "My first blog post!",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}
post

Продолжение в файле [6_MongoDB_client.py](./6_MongoDB_client.py)